# The Threefry random number generator

**Author:** Pierre de Buyl  
This notebook is originally developed in a github
[repository](https://github.com/pdebuyl/compute/blob/master/threefry/threefry.ipynb)
and presented on my [blog](http://pdebuyl.be/blog/2016/threefry-rng.html).
The code is 3-clause BSD and content [CC-BY](https://creativecommons.org/licenses/by/3.0/).

## Threefish

The Threefish block cipher is a part of the encryption candidate [Skein](http://www.skein-hash.info/).
A [block cipher](https://en.wikipedia.org/wiki/Block_cipher) is a reversible transformation of a
sequence of bytes that depends on a user chosen key. Threefish (see the Wikipedia
[page](https://en.wikipedia.org/wiki/Threefish) and the official
[specification](http://www.skein-hash.info/)) is built of several simple components:
  - a mix function that operates on words
  - a permutation function $\pi$ that operates on several consecutive words
  - a key schedule, that defines how the information in the key is applied within the algorithm

Here, a *word* is defined as a sequence of 64 bits. In practice, it is stored as an unsigned 64-bit
integer. All sums are computed modulo $2^{64}-1$.

The full algorithm, going from a series of "plain text" words $p_i$ to the corresponding "cipher" $c_i$
consists in applying the following steps for $N_r$ rounds:
1. one round in four, add the key to the current value of the data
2. apply the mix function to two consecutive words at a time
3. permute the words according to $\pi$

After the last round, a final addition of the key is applied and the full data is XOR-ed with the
initial value $p$. In Threefish, the key schedule is modified by a *tweak*, an extra set of two
words.
The specification provides:
  - a set of rotation constants for the mix function
  - the constant C240 that is part of the key schedule
  - the permutation function $\pi$
  - the number of rounds.

Threefish is specified for blocks of 256, 512 and 1024 bits. Below is a plain implementation of
Threefish-256 that operates on 4 words blocks, in Python (the code is intended for Python 3).
The test reproduces the content of the example found on the official NIST CD release of Skein.
Intermediate steps are obtained by adding the argument "debug=True".

In [ ]:
NW = 4
C240 = 0x1BD11BDAA9FC1A22
N_ROUNDS=72
MASK = 2**64-1
pi = (0, 3, 2, 1)
R_256 = ((14, 16), (52, 57), (23, 40), (5, 37), (25, 33), (46, 12), (58, 22), (32, 32))

def rotl_64(x, d):
    return ((x << d) | (x >> (64-d))) & MASK

def mix(x0, x1, R):
    y0 = (x0+x1) & MASK
    y1 = rotl_64(x1, R) ^ y0
    return y0, y1

def key_schedule(K, TW, s):
    return (K[(s)%(NW+1)] & MASK,
              (K[(s+1)%(NW+1)] + TW[s%3]) & MASK,
              (K[(s+2)%(NW+1)] + TW[(s+1)%3]) & MASK,
              (K[(s+3)%(NW+1)] + s) & MASK)

def threefish(p, K, TW, debug=False):
    K = (K[0], K[1], K[2], K[3], C240^K[0]^K[1]^K[2]^K[3])
    TW = (TW[0], TW[1], TW[0]^TW[1])
    v = list(p)
    for r in range(N_ROUNDS):
        e = [0]*4
        if r%4 == 0:
            ksi = key_schedule(K, TW, r//4)
            for i in range(NW):
                e[i] = (v[i] + ksi[i]) & MASK
            if debug: print('key injection   ', list(map(hex, e)))
        else:
            e = v
        f = [0]*4
        f[0], f[1] = mix(e[0], e[1], R_256[r%8][0])
        f[2], f[3] = mix(e[2], e[3], R_256[r%8][1])
        if (r%2 == 0) and debug: print('end of round %03i' % (r+1), list(map(hex, f)))
        for i in range(NW):
            v[i] = f[pi[i]]
        if (r%2 == 1) and debug: print('end of round %03i' % (r+1), list(map(hex, v)))

    ksi = key_schedule(K, TW, N_ROUNDS//4)
    v = [((x + k)^pp) & MASK for x, k, pp in zip(v, ksi, p)]
    if debug: print(list(map(hex, v)))

    return v


In [ ]:
# Test run with parameters from the NIST CD, file KAT_MCT/skein_golden_kat_internals.txt

K = (0x1716151413121110, 0x1F1E1D1C1B1A1918, 0x2726252423222120, 0x2F2E2D2C2B2A2928)
TW = (0x0706050403020100, 0x0F0E0D0C0B0A0908)

c = threefish((0xF8F9FAFBFCFDFEFF, 0xF0F1F2F3F4F5F6F7, 0xE8E9EAEBECEDEEEF, 0xE0E1E2E3E4E5E6E7), K, TW)
print([hex(x) for x in c])

## Block ciphers, random numbers and Threefry

The motivation for this post (and the corresponding code) is to generate random numbers. The use
of cryptographic protocols as *keyed counter-based pseudorandom number generators* is presented
in Salmon *et al* "Parallel Random Numbers: As Easy as 1, 2, 3", [doi:10.1145/2063384.2063405](http://dx.doi.org/10.1145/2063384.2063405), Proceedings of SC'11.
The use of the the Skein algorithm (based on Threefish) is already mentioned in its specification, but
the paper by Salmon *et al* goes further by specializing the algorithm (and others as well) for use in a
PRNG context. I present here their specialization of Threefish: Threefry.
Threefry is based on the Threefish block cipher, with two shortcuts. The number of rounds is reduced and
the tweak that influence the key schedule is removed. I concentrate here on Threefry-2x64, that is using
two 64-bit words, with 20 rounds.

Whereas the prototypical PRNG steps from integer value to integer value using (more or less) complex
iteration schemes, a counter based PRNG gives a random value directly for any point in the sequence at a
fixed computational cost.
The purpose of a cryptographic algorithm is to make it hard to guess the input from the output. Changing
the input value from $i$ to $i+1$ will output an uncorrelated value.

The algorithm depends on a key of two words. One of them can be used as the seed and the other to identify
one stream (i.e. a compute unit in a parallel computing environment).
The position in the random stream is given as the "plain text" word.

Practically, it means that instead of keeping a state for the PRNG it is possible to request `random_number(counter, key)` where `random_number` is a pure function, `counter` is simply the iteration number that is requested and `key`
is used to identify the stream (including a stream "id" and a seed).

The authors of the paper provide an implementation at http://www.thesalmons.org/john/random123/ or http://www.deshawresearch.com/resources_random123.html

As I had wanted to understand the algorithm and have a working Threefish above, here is a Python version of
Threefry-2x64.

In [ ]:
NW_THREEFRY = 2
C240 = 0x1BD11BDAA9FC1A22
N_ROUNDS_THREEFRY = 20
MASK = 2**64-1
R_THREEFRY = (16, 42, 12, 31, 16, 32, 24, 21)

def key_schedule_threefry(K, s):
    return (K[(s)%(NW_THREEFRY+1)] & MASK,
              (K[(s+1)%(NW_THREEFRY+1)] + s) & MASK)

def threefry(p, K, debug=False):
    K = (K[0], K[1], C240^K[0]^K[1])
    v = list(p)
    for r in range(N_ROUNDS_THREEFRY):
        if r%4 == 0:
            ksi = key_schedule_threefry(K, r//4)
            e = [ (v[0]+ksi[0]) & MASK, (v[1]+ksi[1]) & MASK ]
            if debug: print('key injection   ', list(map(hex, e)))
        else:
            e = v
        v[0], v[1] = mix(e[0], e[1], R_THREEFRY[r%8])
        if debug: print('end of round %03i' % (r+1), list(map(hex, v)))

    ksi = key_schedule_threefry(K, N_ROUNDS_THREEFRY//4)
    v = [(x + k) & MASK for x, k in zip(v, ksi)]
    if debug: print(list(map(hex, v)))

    return v


In [ ]:
print(list(map(hex, threefry((0x0,0x0), (0x0, 0x0)))))

In [ ]:
ff = 0xffffffffffffffff
print(list(map(hex, threefry((ff,ff), (ff, ff)))))

In [ ]:
print(list(map(hex, threefry((0x243f6a8885a308d3, 0x13198a2e03707344), (0xa4093822299f31d0, 0x082efa98ec4e6c89)))))